In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from scipy.interpolate import CubicSpline
import random
from KDEpy import FFTKDE

plt.style.use('default')
plt.rcParams.update({'font.size': 13, 'font.family': 'Helvetica'})

In [ ]:
total_fdps = 100*[None,]
total_fdrs = 100*[None,]

for i in range(100):
    pi0 = 0.3
    big_n = 10000
    neg_n = int(big_n * pi0)
    zs_n = 5000

    random.seed()
    
    xs = np.append(-np.inf*np.ones(neg_n), st.norm.rvs(3, 1, big_n - neg_n))
    ys = st.norm.rvs(0, 1, big_n)
    

    scores = pd.DataFrame(np.array([xs, ys]).T)
    scores.columns = ['x', 'y']

    scores['w'] = [max(el.y, el.x) for el in scores.itertuples()]

    scores['status'] = scores['y'] > scores['x']
    scores.sort_values('w', ascending=True, inplace=True)

    ws = scores['w'].values
    zs = sorted(st.norm.rvs(0, 1, zs_n))

    zs_arr = np.array(zs)
    zs = np.array(zs)
    pvals = np.array([(len(zs_arr[zs_arr >= x])+1) / (len(zs_arr) + 1) for x in scores['w'].values][::-1])
    pi0_est = get_pi0(pvals)
    print(i)

    results = new_ent_mixmax(ws, zs, pi0_est, scores)

    scores.sort_values('w', ascending=False, inplace=True)
    scores['fdp'] = scores['status'].cumsum() / np.arange(1, len(scores)+1)

    total_fdrs[i] = results[1:]
    total_fdps[i] = scores['fdp'][::-1][1:].values


In [ ]:
results = new_ent_mixmax(ws, zs, pi0_est, scores)

scores.sort_values('w', ascending=False, inplace=True)
scores['fdp'] = scores['status'].cumsum() / np.arange(1, len(scores)+1)


plt.plot(results[1:], scores['fdp'][::-1][1:])
plt.xlim(0, 0.1)
plt.ylim(0, 0.13)
plt.plot([0, 0.1], [0, 0.1])

In [ ]:
plot_optimal_fdp_fdr(total_fdps, total_fdrs, "test")

In [ ]:
def get_pi0(pvals, final_lambda=0.95):
    pi0s = []
    lambdas = np.linspace(0, 0.99, 100)

    for l in lambdas:

        pi0_cur = len(pvals[pvals > l]) / (len(pvals) * (1-l))
        pi0s.append(pi0_cur)

    cs = CubicSpline(lambdas, pi0s)
    return cs([final_lambda])[0]


def get_fdps(scores, zs):

    fdps = []

    for threshold in zs:

        new_scores = scores[scores['w'] > threshold]
        cur_fdp = len(new_scores[new_scores['status'] == True]) / len(new_scores)
        fdps.append(cur_fdp)
    
    return fdps


def ent_mixmax(ws, zs, pi0, nwz, nzz, nzw, nww):

    n = len(ws)
    n_e = len(zs)

    fg = n / n_e

    e1 = 0
    j = n_e - 1

    rs = np.zeros(n_e)

    for m in range(n_e-1, 0, -1):

        p_est = (nwz[m] / ((1-pi0) * nzz[m])) - ((fg * pi0) / (1 - pi0))
        e1 += min(1, max(0, p_est)) * (1- pi0)
        
        rs[m] = ( pi0 * fg * nzw[m] + fg * e1) / nww[m]
        rs[m] = min(rs[m], 1)

    return rs


def new_ent_mixmax(ws, zs, pi0, scores):


    nwz = np.sum(scores['w'].values[:, np.newaxis] <= zs, axis=0)
    nzz = np.sum(zs[:, np.newaxis] <= zs, axis=0)
    nzw = np.sum(zs[:, np.newaxis] > ws, axis=0)
    nww = np.sum(scores['w'].values[:, np.newaxis] >= ws, axis=0)

    n = len(ws)
    n_e = len(zs)

    fg = n / n_e

    e1 = 0
    j = n_e - 1

    rs = np.zeros(n)

    for m in range(n-1, 0, -1):

        while (j > 0) and (zs[j] >= ws[m]):

            p_est = (nwz[j] / ((1-pi0) * nzz[j])) - ((fg * pi0) / (1 - pi0))
            e1 += min(1, max(0, p_est)) * (1- pi0)
            j -= 1
        
        rs[m] = ( pi0 * fg * nzw[m] + fg * e1) / nww[m]
        rs[m] = min(rs[m], 1)

    return rs

In [ ]:
def plot_optimal_fdp_fdr(total_fdps, total_fdrs, outname):

    fdrs = total_fdrs
    fdps = total_fdps
    fdr_bins = np.linspace(0.001, 0.1, 100)

    bin_results = AveragedFdpFdr(fdr_bins).put_in_bins(fdrs, fdps)
    cis_to_plot = ConfidenceInterval().get_mean_cis(bin_results, 0, 1, 0.68)
    plot_mean_cis_fdp_fdr(fdr_bins, cis_to_plot, outname)


def plot_mean_cis_fdp_fdr(fdr_bins, stats, outname):

    fig, ax = plt.subplots(figsize=(7, 5))
    ax.plot(fdr_bins[:-1], stats[:, 1][:-1])
    ax.fill_between(fdr_bins[:-1], stats[:, 0][:-1], stats[:, 2][:-1], color='royalblue', alpha=0.2)
    ax.plot([0, 0.1], [0, 0.1], linestyle='--', color='grey')
    
    ax.set_xlabel("FDR")
    ax.set_ylabel("FDP")
    fig.savefig(f"./{outname}_fdp_fdr.pdf", dpi=800, bbox_inches="tight")



class AveragedFdpFdr:

    def __init__(self, fdr_bins) -> None:
        
        self.fdr_bins = fdr_bins
        self.range_bins = range(len(fdr_bins))


    def put_in_bins(self, fdrs, fdps):

        fdr_digitized = [np.digitize(fdr_arr, bins=self.fdr_bins) for fdr_arr in fdrs]
        zipped_fdr_bin_fdp = [np.array(list(zip(fdr_digitized[idx], fdps[idx]))) for idx in range(len(fdps))]
        results = [self.separate_zipped(x) for x in zipped_fdr_bin_fdp]
        extracted = [self.extract_from_each_bin(results, idx) for idx in self.range_bins]
        aggregated_for_each_bin = [self.aggregate_for_each_bin(extracted, idx)[0] for idx in self.range_bins]

        return aggregated_for_each_bin


    @staticmethod
    def aggregate_for_each_bin(extracted, bin_idx):
        return [[item for array in extracted[bin_idx] for item in array]]

    @staticmethod
    def extract_from_each_bin(results, idx):
        return [results[item][idx] for item in range(len(results))]


    def separate_zipped(self, zipped):
        return [zipped[zipped[:, 0] == idx + 1][:, 1] for idx in self.range_bins]

  
class ConfidenceInterval:

    def __init__(self) -> None:
        pass

    def get_mean_cis(self, bin_results, l_cutoff, u_cutoff, confidence_level):

        truncated = [x[int(l_cutoff * len(x)): int(u_cutoff * len(x))] for x in bin_results]
        stats = np.array([self.calculate_confidence_interval(x, confidence_level) for x in truncated])

        return stats


    def calculate_confidence_interval(self, sample, conf_level):

        sample_mean = np.mean(sample)
        margin_of_error = self.calculate_margin_of_error(sample, conf_level)

        return sample_mean - margin_of_error, sample_mean, sample_mean + margin_of_error


    @staticmethod
    def calculate_margin_of_error(sample, conf_level):
        sample_std = np.std(sample, ddof=1)
        alpha = 1 - conf_level
        critical_value = st.t.ppf((1 - alpha) / 2, df=len(sample))
        std_error = sample_std / np.sqrt(len(sample))
        margin_of_error = critical_value * std_error
        return margin_of_error